<a href="https://colab.research.google.com/github/jiheddachraoui/P2M/blob/main/radar2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive')
!git clone https://github.com/jiheddachraoui/radar_cnn.git

fatal: destination path 'radar_cnn' already exists and is not an empty directory.


In [4]:
import sys    
path = '/content/radar_cnn'

sys.path.append(path)

# link with Git

In [ ]:
!git config --global user.email "jiheddachraoui@outlook.fr"
!git config --global user.name "jiheddachraoui"
!git config --global user.password "xxxxx"

In [ ]:
token = 'your_token'
username = 'jiheddachraoui'
repo = 'radar_cnn'
!git clone https://{token}@github.com/{username}/{repo}



Cloning into 'radar_cnn'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 84 (delta 37), reused 29 (delta 12), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [ ]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


**push**

In [ ]:
%cd {repo}
!git add --all

/content/radar_cnn


In [ ]:
!git commit -m "first c"
%ls

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean
bash/                dataset/  gen_dataset.py    LICENSE    RTCnet/
cylinder_cluster.py  figures/  instance_seg.py*  README.md


In [ ]:
!git push

Everything up-to-date


**pull**

In [ ]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


# Generate DATASET

In [5]:
import os
import os.path as osp 
import json

import numpy as np 

def create_path():
    """
    Create the path for the dataset.
    
    Parameters
    ----------
    None
    
    Returnes
    --------
    dataset_DIR: string
                 The path to the dataset folder
    data_DIR: string
              The path to the data parent folder
    """
    #BASE_DIR = os.path.dirname(os.path.abspath(__file__))
    #BASE_DIR = "/content/RTCnet"
    BASE_DIR = "/content/radar_cnn"
    dataset_DIR = osp.join(BASE_DIR, 'dataset')
    data_DIR = osp.join(dataset_DIR, 'data')
    train_DIR = osp.join(data_DIR, 'train')
    test_DIR = osp.join(data_DIR, 'test')
    val_DIR = osp.join(data_DIR, 'val')
    if not osp.exists(train_DIR):
        os.makedirs(train_DIR)
    if not osp.exists(test_DIR):
        os.makedirs(test_DIR)
    if not osp.exists(val_DIR):
        os.makedirs(val_DIR)
    return dataset_DIR, data_DIR


def generate_dataset_single(num_features, num_samples):
    
    """
    Generate each single dataset
    
    Parameters
    ----------
    num_features: int
                  The number of features for each sample
    num_samples: int
                 The number of samples for the dataset
    """
    features_x = 120 * np.random.rand(num_samples,1) 
    features_y = 100 * np.random.rand(num_samples,1) - 50
    features_v = 30 * np.random.rand(num_samples,1) - 15
    features_rcs = 5 * np.random.rand(num_samples,1) 
    features_low_level = 3000 * np.random.rand(num_samples, num_features - 4)
    features = np.concatenate([features_x, features_y, features_v, features_rcs, features_low_level], axis=1)   
    labels = np.random.randint(4,size=(num_samples)) 
    num_targets_per_frame = 20
    num_frames = num_samples / num_targets_per_frame
    frame_id = np.arange(num_frames + 1)
    frame_id = np.repeat(frame_id, num_targets_per_frame)
    frame_id = frame_id[:num_samples]
    instance_id = labels = np.random.randint(6,size=(num_samples)) 

    return features, labels, frame_id, instance_id


if __name__ == "__main__":
    dataset_DIR, data_DIR = create_path()
    num_features = 804
    num_samples_train = 10000
    num_samples_val = 1000
    num_samples_test = 4000
    features_train, labels_train, frame_id_train, instance_id_train  = generate_dataset_single(num_features, num_samples_train)
    features_val, labels_val, frame_id_val, instance_id_val = generate_dataset_single(num_features, num_samples_val)
    features_test, labels_test, frame_id_test, instance_id_test = generate_dataset_single(num_features, num_samples_test)
    
    """
    The meta data is used for configuration of the project. 
    Most attributes are related to the real data generator that generates data from real files recorded by the vehicle.
    For example, polar_coord means whether the coordinate of radar targets will be transformed to Cartesian coordinate system.
    """
    meta_data = {

    "all_mirroring": True,
    "attribute": "all data with FOV filtering 32 cropping",
    "augmentation": True,
    "crop_window_size": 5,
    "filter_lowlevel": True,
    "mirroring": True,
    "nms": True,
    "polar_coord": True,
    "shuffle": True,
    "test_only_nearby": False,
    "use_manual_annotation": True,
    "view_cluster": True,
    "viz_input": False,
    "x_lim": 1000,
    "yaw_threshold": 100
    }

    with open(osp.join(dataset_DIR, 'meta_data.json'), 'w') as fp:
        json.dump(meta_data, fp, sort_keys=True, indent=4, separators=(',', ': '))

    np.save(osp.join(data_DIR, 'train','features.npy'), features_train)
    np.save(osp.join(data_DIR, 'train','labels.npy'), labels_train)
    np.save(osp.join(data_DIR, 'train','frame_id.npy'), frame_id_train)
    np.save(osp.join(data_DIR, 'val','features.npy'), features_val)
    np.save(osp.join(data_DIR, 'val','labels.npy'), labels_val)
    np.save(osp.join(data_DIR, 'val','frame_id.npy'), frame_id_val)
    np.save(osp.join(data_DIR, 'test','features.npy'), features_test)
    np.save(osp.join(data_DIR, 'test','labels.npy'), labels_test)
    np.save(osp.join(data_DIR, 'test','frame_id.npy'), frame_id_test)
    np.save(osp.join(data_DIR,'instance_id_test.npy'), instance_id_test)

# train


In [6]:
# Python Library
import os
import os.path as osp 
import sys 
from time import time
import json
import argparse
from copy import deepcopy
from datetime import datetime

# Third-party library
import numpy as np 
import matplotlib.pyplot as plt 
import cv2
import easydict
# Pytorch
import torch 
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader

# Network 
from RTCnet.RTCnet import RTCnet, TargetMLP
from RTCnet.RTCnet_utils import Trainer
from RTCnet.TargetLoader import TargetModeDataset, ToTensor, Permutation

#BASE_DIR = os.path.dirname(os.path.abspath(__file__)) # Base directory of the RTC module
BASE_DIR = "/content/radar_cnn/RTCnet"

def train_ova(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            chose_class 
            ):
    """ 
        Function to train one-over-all model
    Args:
        train_data (TargetModeDataset): The training dataset
        val_data (TargetModeDataset): The validation dataset
        train_loader (DataLoader): The dataloader for training dataset
        val_loader (DataLoader): The dataloader for validation dataset
        weights_all (NumpyArray): Normalized weights for classes
        model (RTCnet): The network modules
        optimizer (Optimizer): The optimizer in torch.optim
        scheduler: The scheduler for training
        eval_frequency (int): The frequency used for evaluation by validation set. 0 means evaluating after each epoch
        use_gpu (bool): Whether using GPU for training and validation
        lr_decay_step (int): the number of steps for learning rate decay
        decay_f (double): the decay factor of learning rate
        lr_clip (double): the clip of learning rate during decay process
        result_folder (string): the folder to save the training result
        n_epochs (int): the number of epochs for training
        chose_class (int): the class number for the chosen class  

    """
    class_list = ['others', 'ped', 'biker', 'car']
    print("start training {} vs All".format(class_list[chose_class]))
    append_str = "{}_vs_All".format(class_list[chose_class])
    weights = torch.tensor(np.array([np.sum(weights_all)-weights_all[chose_class], weights_all[chose_class]]))
    if use_gpu:
        weights = weights.cuda()
    print("weights:{}".format(weights))
    loss_func = nn.CrossEntropyLoss(weight = weights)
    #################### data ##########################
    train_data.to_ova(chose_class = chose_class)
    val_data.to_ova(chose_class = chose_class)
    ################### Trainer ##########################
    trainer = Trainer(
                model, 
                loss_func,
                optimizer,
                lr_scheduler = scheduler,
                eval_frequency = eval_frequency,
                use_gpu = use_gpu,
                lr_decay_step=lr_decay_step,
                lr_decay_f=decay_f,
                lr_clip=lr_clip,
                save_checkerpoint_to=result_folder,
                append_str= append_str
    )
    trainer.train(
            n_epochs,
            train_loader,
            val_loader=val_loader,
            best_loss=1e5,
            start_it=0
    )
    loss_trajectory = trainer.trace_loss
    loss_trajectory_train = trainer.trace_loss_train
    np.save(osp.join(result_folder, "loss_trajectory" + append_str), loss_trajectory)
    np.save(osp.join(result_folder, "train_loss_trajectory" + append_str), loss_trajectory_train)

def train_ovo(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            class_positive,
            class_negative
            ):
    """ 
        Function to train one-over-one model

    Args:
        train_data (TargetModeDataset): The training dataset
        val_data (TargetModeDataset): The validation dataset
        train_loader (DataLoader): The dataloader for training dataset
        val_loader (DataLoader): The dataloader for validation dataset
        weights_all (NumpyArray): Normalized weights for classes
        model (RTCnet): The network modules
        optimizer (Optimizer): The optimizer in torch.optim
        scheduler: The scheduler for training
        eval_frequency (int): The frequency used for evaluation by validation set. 0 means evaluating after each epoch
        use_gpu (bool): Whether using GPU for training and validation
        lr_decay_step (int): the number of steps for learning rate decay
        decay_f (double): the decay factor of learning rate
        lr_clip (double): the clip of learning rate during decay process
        result_folder (string): the folder to save the training result
        n_epochs (int): the number of epochs for training
        class_positive(int): the class number for the positive class
        class_negative(int): the class number for the negative class

    """
    class_list = ['others', 'ped', 'biker', 'car']
    print("start training {} vs {}".format(class_list[class_positive], class_list[class_negative]))
    append_str = "{}_vs_{}".format(class_list[class_positive], class_list[class_negative])
    weights = torch.tensor(np.array([weights_all[class_negative], weights_all[class_positive]]))
    if use_gpu:
        weights = weights.cuda()
    print("weights:{}".format(weights))
    loss_func = nn.CrossEntropyLoss(weight = weights)
    #################### data ##########################
    train_data.to_ovo(class_positive, class_negative)
    val_data.to_ovo(class_positive, class_negative)
    ################### Trainer ##########################
    trainer = Trainer(
                model, 
                loss_func,
                optimizer,
                lr_scheduler = scheduler,
                eval_frequency = eval_frequency,
                use_gpu = use_gpu,
                lr_decay_step=lr_decay_step,
                lr_decay_f=decay_f,
                lr_clip=lr_clip,
                save_checkerpoint_to=result_folder,
                append_str= append_str
    )
    trainer.train(
            n_epochs,
            train_loader,
            val_loader=val_loader,
            best_loss=1e5,
            start_it=0
    )
    loss_trajectory = trainer.trace_loss
    np.save(osp.join(result_folder, "loss_trajectory" + append_str), loss_trajectory)
    loss_trajectory_train = trainer.trace_loss_train
    np.save(osp.join(result_folder, "train_loss_trajectory" + append_str), loss_trajectory_train)



def main():
    parser = argparse.ArgumentParser()
    '''parser.add_argument("--data", type=str, default = osp.join(BASE_DIR, osp.pardir, 'dataset','data'), help = "The data path that contains train, val and test folder, this is generated by gen_input_data_and_baseline.py" )
    parser.add_argument("--batch_size", type=int, default = 1024, help = "batch size for training" ) 
    parser.add_argument("--rm_speed", type=bool, default = False, help = "Whether to remove the speed during training" )
    parser.add_argument("--rm_rcs", type=bool, default = False, help = "Whether to remove the RCS value during training" ) 
    parser.add_argument("--n_epochs", type=int, default = 1, help = "number of epochs" ) '''
    #args            = parser.parse_args()
    #args = parser.parse_args(argv[1:])
    args = easydict.EasyDict({
        "data": '/content/radar_cnn/dataset/data',
        "batch_size": 1024,
        "rm_speed": False,
        "rm_rcs": False,
        "n_epochs": 1
    
})
    data_path       = args.data
    rm_speed        = args.rm_speed
    rm_rcs          = args.rm_rcs 
    n_epochs        = args.n_epochs
    model_version   = 4
    chosen_feature_type = 'high' if model_version == 3 else 'low' 

    # Get the meta information of the dataset
    data_meta_dir   = osp.join(data_path, os.pardir)
    data_meta_path  = osp.join(data_meta_dir, 'meta_data.json')
    with open(data_meta_path) as fp:
        data_meta = json.load(fp)
    data_x_lim      = data_meta['x_lim']                    # The maximum limit for the longitudinal distance

    # Training setup
    lr_start        = 1e-3                                  # The leraning rate starts from lr_start, but it decays according to certain policies
    eval_frequency  = 0                                     # The frequency used for evaluation by validation set. 0 means evaluating after each epoch
    batch_size      = args.batch_size                       # The batch size for stochastic gradient descent
    lr_decay_step   = 2000                                  # The step after which the learning rate is decayed. But if the optimizer is used, this is useless.
    decay_f         = 0.9                                   # The decay factor for the learning rate. After each lr_decay_step iterations, the learning rate is equal to decay_f * last leraning rate
    lr_clip         = 2e-4                                  # The clip of learning rate. The learning rate stops decaying after this value
    use_gpu         = False                                 # Whether use gpu to train
    dropout         = True
    if_shuffle      = True                                  # Whether to shuffle the data when loading them. This should always be true when training
    speed_limit     = 0                                     # The speed limit used when loading data on the fly. But if the data is already filtered when generated, this can be set as 0
    high_dims       = 4                                     # The number of dimensions for high-level features
    dist_near       = 0                                     # The nearest distance threshold. If using polar coordinate, this is range rather than distance
    binary_class    = False                                 # If using binary class
    use_weight      = True                                  # If using weights for loss function
    weights_factor  = np.array([1/0.5, 1/1, 1/0.5, 1/1])    # The scaling factor for the weights, considering unbalanced classes
    rm_cars         = False                                 # Whether to remove cars when training
    input_size      = 5                                     # The input size of RTC2 window
    rm_position     = False                                 # Whether to remove position during training

    t = int(time())
    result_parent_path = osp.join(BASE_DIR, osp.pardir, 'results')

    result_folder = osp.join(result_parent_path,"RTCresults", str(t))
    train_info_path = osp.join(result_parent_path, 'RTCtrain_info')

    if not osp.exists(train_info_path):
        os.makedirs(train_info_path)
    if not osp.exists(result_folder):
        os.makedirs(result_folder)

    ################### transforms ###################
    to_tensor = ToTensor()
    perm_position = Permutation()
    composed_trans = transforms.Compose([perm_position, to_tensor])

    ################### data loader ###################  
    ## Train dataset 
    train_data = TargetModeDataset(
                data_path, composed_trans, 
                mode='train', high_dims = high_dims, 
                normalize = True, feature_type = chosen_feature_type,
                norms_path=result_folder,
                speed_limit=speed_limit,
                dist_near= dist_near,
                binary_class = binary_class,
                dist_far=data_x_lim,
                rm_cars=rm_cars,
                rm_position=rm_position,
                rm_speed = rm_speed,
                rm_rcs=rm_rcs)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=if_shuffle, num_workers=2)
    ## Validation dataset
    val_data = TargetModeDataset(
                data_path, composed_trans, 
                mode='val', high_dims=high_dims, 
                normalize=True, feature_type= chosen_feature_type,
                norms_path=result_folder,
                speed_limit=speed_limit,
                dist_near= dist_near,
                binary_class = binary_class,
                dist_far=data_x_lim,
                rm_cars=rm_cars,
                rm_position=rm_position,
                rm_speed = rm_speed,
                rm_rcs=rm_rcs)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)

    train_data.save_features_labels()
    val_data.save_features_labels()

    ################### Model ################### 

    if model_version == 4:
        model = RTCnet(
                num_classes=2, 
                Doppler_dims=32, 
                high_dims = high_dims, 
                dropout= dropout,
                input_size = input_size)
    elif model_version == 3:
        model = TargetMLP(
                num_classes=2,
                high_dims=high_dims)

    model.double()


    ################### weight initialization ################### 
    if use_weight:
        num_others = np.sum(train_data.labels==0)
        num_ped = np.sum(train_data.labels == 1)
        num_bikers = np.sum(train_data.labels==2)
        num_car = np.sum(train_data.labels==3)
        print("num_others:{}, num_peds:{}, num_bikers:{}, num_car:{}".format(num_others, num_ped, num_bikers, num_car))

        weights = np.array([1/num_others,1/num_ped,1/num_bikers,1/num_car])
    else:
        weights = np.ones(4)

    weights = np.multiply(weights_factor, weights)
    weights_save = deepcopy(weights.tolist())
    weights_all = np.divide(weights, np.sum(weights))

    ################### Save the training information #############################
    train_info = {
    "data_path": data_path, 
    "lr_start": lr_start,
    "n_epochs": n_epochs,
    "eval_frequency": eval_frequency,
    "batch_size": batch_size,
    "lr_decay_step":lr_decay_step,
    "decay_f":decay_f,
    "lr_clip":lr_clip,
    "use_gpu":use_gpu,
    "t":t,
    "result_folder":result_folder,
    "weights": weights_save,
    "weights_factor": weights_factor.tolist(),
    "if_shuffle":if_shuffle,
    "dropout":dropout,
    "speed_limit": speed_limit,
    "dist_limit": dist_near,
    "binary": binary_class,
    "input_size": input_size,
    "model_version": model_version,
    "data_x_limit": data_x_lim,
    "use_weight":use_weight,
    "rm_cars": rm_cars,
    "rm_position": rm_position,
    "rm_speed": rm_speed,
    "rm_rcs": rm_rcs
    }
    with open(osp.join(train_info_path, '{}.json'.format(t)), 'w') as fp:
        print("save the file at :{}".format(osp.join(train_info_path, '{}.json'.format(t))))
        json.dump(train_info, fp, sort_keys=True, indent=4, separators=(',', ': '))




    ################### Optimizer ########################
    optimizer = torch.optim.Adam(model.parameters(), lr=lr_start)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=lr_decay_step,gamma=0.1)
    
    ################### Ped vs ALL ########################
    
    train_ova(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            chose_class = 1 
            )

    ################### Biker VS ALL ######################
    train_ova(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            chose_class = 2
    ) 
    ################### Car VS ALL ######################
    train_ova(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            chose_class = 3
    ) 
    ################### others VS ALL ######################

    train_ova(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            chose_class = 0
    ) 

    ################### Ped VS biker ########################
    train_ovo(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            1,
            2
            )

    #################### Ped VS car #########################
    train_ovo(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            1,
            3
            )
    ################### Biker VS car #######################
    train_ovo(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            2,
            3
            )
    #################### Others VS ped #####################
    train_ovo(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            0,
            1
            )
    #################### Others VS biker #####################
    train_ovo(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            0,
            2
            )
    ##################### Others VS car #######################
    train_ovo(train_data, 
            val_data, 
            train_loader,
            val_loader,
            weights_all, 
            model, 
            optimizer, 
            scheduler, 
            eval_frequency, 
            use_gpu, 
            lr_decay_step, 
            decay_f, 
            lr_clip, 
            result_folder,
            n_epochs,
            0,
            3
            )



if __name__ == "__main__":
    main()
    

train feature shape:(10000, 804) 
train features after removing static targets:(10000, 804)
train features after removing neaby and far away targets:(10000, 804)
val feature shape:(1000, 804) 
val features after removing static targets:(1000, 804)
val features after removing neaby and far away targets:(1000, 804)
num_others:1675, num_peds:1660, num_bikers:1642, num_car:1630
save the file at :/content/radar_cnn/RTCnet/../results/RTCtrain_info/1651703022.json
start training ped vs All
weights:tensor([0.8340, 0.1660], dtype=torch.float64)


initial_validation_loss:0.691807761410452


epochs:   0%|          | 0/1 [00:08<?, ?it/s]
                                                                              
epochs:   0%|          | 0/1 [00:09<?, ?it/s]

validation loss is:0.5302046903642232



epochs: 100%|██████████| 1/1 [00:09<00:00,  9.32s/it]

                                                                     

...

start training others vs car
weights:tensor([0.1691, 0.3291], dtype=torch.float64)


initial_validation_loss:0.7017509786454806


epochs:   0%|          | 0/1 [00:03<?, ?it/s]
                                                                           
epochs:   0%|          | 0/1 [00:03<?, ?it/s]

validation loss is:0.6974144880584349



epochs: 100%|██████████| 1/1 [00:03<00:00,  3.59s/it]

                                                                    

# test

In [7]:

# Python Libraries
import os 
import sys 
import os.path as osp 
import argparse

# Pytorch
import torch 
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn 

# Third-party libraries
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import json 
import easydict

# Modules from current project
from RTCnet.TargetLoader import TargetModeDataset, ToTensor
from RTCnet.RTCnet import RTCnet, TargetMLP
from RTCnet.RTCnet_utils import Tester, Tester_ensemble

#BASE_DIR = os.path.dirname(os.path.abspath(__file__)) # Base directory of the RTC module
BASE_DIR ='/content/radar_cnn/RTCnet'
result_DIR = osp.join(BASE_DIR, osp.pardir, 'results', 'RTCtrain_info')
config_list = os.listdir(result_DIR)
config_list.sort()
parser = argparse.ArgumentParser()
    
#args            = parser.parse_args()
#args = parser.parse_args(argv[1:])
args = easydict.EasyDict({
        "config": osp.join(result_DIR, config_list[-1]),
        "batch_size": 1024,
        "test": None,
        
    
})


def test_ova(
    chose_class,
    result_folder,
    model,
    test_data,
    weights_all,
    test_loader,
    use_gpu
):
    """
    Test the trained model on testing dataset
    
    Parameters
    ----------
    chose_class: int
                 The class index which is chosen for one-over-all model testing
    result_folder: string 
                   The path in which the trained result is saved
    model: RTCnet
           The RTC network 
    test_data: TargetModeDataset 
               The dataset for testing
    weights_all: Array 
                 The class weights 
    test_loader: DataLoader
                 The dataloader for testing dataset 
    use_gpu: bool 
            flag to decide whether GPU is used for acceleration
            
    Returns: Array shape(n_samples, n_classes)
            The scores for each class
    """
    class_list = ['others', 'ped', 'biker', 'car']
    print("start training {} vs All".format(class_list[chose_class]))
    append_str = "{}_vs_All".format(class_list[chose_class])
    model_path      = osp.join(result_folder,'best_checkerpoint_{}.pth'.format(append_str))
    model.load_state_dict(torch.load(model_path))
    model.eval()

    weights = torch.tensor(np.array([np.sum(weights_all) - weights_all[1], weights_all[1]])) if not use_gpu else \
         torch.tensor(np.array([np.sum(weights_all) - weights_all[1], weights_all[1]])).cuda()
    loss_func = nn.CrossEntropyLoss(weights)

    tester = Tester_ensemble(
        model,
        loss_func = loss_func,
        use_gpu=use_gpu
    )
    scores_ova = tester.test(test_loader)
    return scores_ova

def test_ovo(
    class_positive,
    class_negative,
    result_folder,
    model,
    test_data,
    weights_all,
    test_loader,
    use_gpu,
):
    """
    Test the trained model on testing dataset
    
    Parameters
    ----------
    class_positive: int
                    The class index which is chosen for positive class (one-vs-one)
    class_negative: int
                    The class index which is chosen for negative class (one-vs-one)
    result_folder: string 
                   The path in which the trained result is saved
    model: RTCnet
           The RTC network 
    test_data: TargetModeDataset 
               The dataset for testing
    weights_all: Array 
                 The class weights 
    test_loader: DataLoader
                 The dataloader for testing dataset 
    use_gpu: bool 
            flag to decide whether GPU is used for acceleration
            
    Returns: Array shape(n_samples, n_classes)
            The scores for each class
    """
    class_list = ['others', 'ped', 'biker', 'car']
    print("start training {} vs {}".format(class_list[class_positive], class_list[class_negative]))
    append_str = "{}_vs_{}".format(class_list[class_positive], class_list[class_negative])
    weights = torch.tensor(np.array([weights_all[class_negative], weights_all[class_positive]]))
    if use_gpu:
        weights = weights.cuda()
    print("weights:{}".format(weights))
    loss_func = nn.CrossEntropyLoss(weight = weights)

    model_path      = osp.join(result_folder,'best_checkerpoint_{}.pth'.format(append_str))
    model.load_state_dict(torch.load(model_path))
    model.eval()

    tester = Tester_ensemble(
        model,
        loss_func = loss_func,
        use_gpu=use_gpu
    )
    scores_ovo = tester.test(test_loader)
    return scores_ovo

def main():
    cfg_path        = args.config
    test_data_path  = args.test

    with open(cfg_path) as fp:
        cfg = json.load(fp)
    data_path       = cfg["data_path"]
    use_gpu         = cfg['use_gpu']
    result_folder   = cfg['result_folder']
    weights_all     = cfg['weights']
    use_set         = ["train", "val", "test"]
    weights_factor  = cfg["weights_factor"]
    dropout         = cfg['dropout']
    weights_factor  = np.array(weights_factor)
    weights_all     = np.array(weights_all)
    binary_class    = cfg['binary']
    input_size      = cfg['input_size']
    rm_cars         = cfg["rm_cars"]
    speed_limit     = cfg["speed_limit"]
    dist_near       = cfg["dist_limit"]
    dist_far        = cfg["data_x_limit"]
    if "rm_speed" in cfg:
        rm_speed = cfg["rm_speed"]
    else:
        rm_speed = False
    if "rm_rcs" in cfg:
        rm_rcs = cfg["rm_rcs"]
    else:
        rm_rcs = False
    test_with_nearby = False
    only_slow = False
    only_fast = False
    if "rm_position" in cfg:
        rm_position = cfg['rm_position']
    else:
        rm_position = False
    
    use_nearby = False
    if use_nearby or test_with_nearby:
        dist_far = 60
    if rm_position:
        high_dims = 2
        dist_far = 100
    else:
        high_dims = 4
    if test_data_path is None:
        test_data_path = data_path
    if rm_cars:
        num_classes = 3
    elif binary_class:
        num_classes = 2
    else:
        num_classes = 2
    weights_all = np.multiply(weights_factor, weights_all)
    model_version   = 4
    chosen_feature_type = 'high' if model_version == 3 else 'low'
    cfg['testset'] = test_data_path
    with open(osp.join(result_folder, 'info.json'), 'w') as fp:
        json.dump(cfg, fp, sort_keys=True, indent=4, separators=(',', ': '))
    
    batch_size = args.batch_size

    ################### transforms ###################
    to_tensor = ToTensor()
    composed_trans = transforms.Compose([to_tensor])

    test_data = TargetModeDataset(
                test_data_path, composed_trans, 
                mode='test', high_dims=high_dims, 
                normalize=True, feature_type= chosen_feature_type,
                norms_path=result_folder,
                speed_limit=speed_limit,
                dist_near=dist_near,
                binary_class=binary_class,
                dist_far=dist_far,
                rm_cars=rm_cars,
                rm_position=rm_position,
                zero_low=False,
                only_slow=only_slow,
                only_fast=only_fast,
                rm_speed=rm_speed,
                rm_rcs=rm_rcs)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=1)
    np.save(osp.join(result_folder, "valid_indx_test"), test_data.indx_valid)

    ################### Define model ###################
    if model_version == 4:
        model = RTCnet(
                num_classes=2, 
                Doppler_dims=32, 
                high_dims = high_dims, 
                dropout= dropout,
                input_size = input_size)
    elif model_version == 3:
        model = TargetMLP(
                num_classes=2,
                high_dims=high_dims)

    ################### Ped VS ALL ###########
    scores_Ped_vs_All = test_ova(
                        1,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Biker_vs_All = test_ova(
                        2,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Car_vs_All = test_ova(
                        3,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Others_vs_All = test_ova(
                        0,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Ped_vs_Biker = test_ovo(
                        1,
                        2,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Ped_vs_Car = test_ovo(
                        1,
                        3,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )
    scores_Biker_vs_Car = test_ovo(
                        2,
                        3,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Others_vs_Ped = test_ovo(
                        0,
                        1,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Others_vs_Biker = test_ovo(
                        0,
                        2,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    scores_Others_vs_Car = test_ovo(
                        0,
                        3,
                        result_folder,
                        model,
                        test_data,
                        weights_all,
                        test_loader,
                        use_gpu
                    )

    

    final_score1 = np.reshape(scores_Ped_vs_Biker * (scores_Ped_vs_All + scores_Biker_vs_All) \
                    + scores_Ped_vs_Car * (scores_Ped_vs_All + scores_Car_vs_All) \
                    + (1 - scores_Others_vs_Ped) * (scores_Ped_vs_All + scores_Others_vs_All), (-1, 1))
    final_score2 = np.reshape(( 1 - scores_Ped_vs_Biker) * (scores_Biker_vs_All + scores_Ped_vs_All)\
                    + scores_Biker_vs_Car * (scores_Biker_vs_All + scores_Car_vs_All)\
                    + (1 - scores_Others_vs_Biker) * (scores_Biker_vs_All + scores_Others_vs_All), (-1, 1))
    final_score3 = np.reshape((1 - scores_Ped_vs_Car) * (scores_Car_vs_All + scores_Ped_vs_All)\
                    + (1-scores_Biker_vs_Car) * (scores_Car_vs_All + scores_Biker_vs_All)\
                    + (1-scores_Others_vs_Car) * (scores_Car_vs_All + scores_Others_vs_All), (-1, 1))
    final_score0 = np.reshape(scores_Others_vs_Ped * (scores_Others_vs_All + scores_Ped_vs_All)\
                    + scores_Others_vs_Biker * (scores_Others_vs_All + scores_Biker_vs_All)\
                    + scores_Others_vs_Car * (scores_Others_vs_All + scores_Car_vs_All), (-1, 1))
    final_score = np.concatenate([final_score0, final_score1, final_score2, final_score3], axis=1)
    pred_labels_all = np.argmax(final_score, axis=1)
    true_labels_all = test_data.labels

    np.save(osp.join(result_folder, "final_score.npy"), final_score)
    np.save(osp.join(result_folder, "true_labels_test.npy"), true_labels_all)


    np.save(osp.join(result_folder, "pred_labels_{}".format("test")), pred_labels_all)
    ################### Confusion matrix ######################
    class_names = np.array(['Others', 'Ped.', 'Biker', 'Car'])
    confusion_mat = confusion_matrix(true_labels_all, pred_labels_all)
    f1_score_all = f1_score(true_labels_all, pred_labels_all, average='macro')
    f1_score_individual = f1_score(true_labels_all, 
                            pred_labels_all, average=None)
    with open(osp.join(result_folder, "f1score_{}.txt".format("test")),'a') as f:
        np.savetxt(f,np.reshape(f1_score_all, (1,-1)),fmt='%f')
        np.savetxt(f,np.reshape(f1_score_individual, (1,-1)),fmt='%f')
    np.savetxt(osp.join(result_folder, '{}.txt'.format("test")),confusion_mat,fmt='%f')
    np.save(osp.join('{}.npy'.format("test")),confusion_mat)
if __name__ == "__main__":
    main()



test feature shape:(4000, 804) 
test features after removing static targets:(4000, 804)
test features after removing neaby and far away targets:(4000, 804)
start training ped vs All


start training biker vs All


start training car vs All


start training others vs All


start training ped vs biker
weights:tensor([0.0024, 0.0006], dtype=torch.float64)


start training ped vs car
weights:tensor([0.0006, 0.0006], dtype=torch.float64)


start training biker vs car
weights:tensor([0.0006, 0.0024], dtype=torch.float64)


start training others vs ped
weights:tensor([0.0006, 0.0024], dtype=torch.float64)


start training others vs biker
weights:tensor([0.0024, 0.0024], dtype=torch.float64)


start training others vs car
weights:tensor([0.0006, 0.0024], dtype=torch.float64)


# instance_seg

In [8]:
import sys 
import os 
import os.path as osp 
import numpy as np 
import json 
from sklearn.cluster import DBSCAN
import tqdm
import matplotlib.pyplot as plt 
from scipy.spatial import distance
from copy import deepcopy
from cylinder_cluster import cylinder_cluster
""" 
Do some work on object level
"""


def cal_precision_recall_single(labels_true, labels_pred, instance_id_true, instance_id_pred, class_label=0, targets_for_debug=None):

    """
    Parameters
    ------------- 
    This function deals with single frame precision and recall calculation
    labels_true: 1-d array
        The true label of the targets of one single frame
    labels_pred: 1-d array
        The predicted label of the targets of one single frame
    instance_id_true: 1-d array
        The instance ID of the targets of one single frame from SSD bounding box
    instance_id_pred: 1-d array
        The instance ID of the post-clustering output
    class_label: an int
        The label of the class to calculation. 0 for background, 1 for pedestrian, 2 for cyclist, 3 for car
    
    Returns
    ------------
    num_TP: an int
        The number of true positives of the class_label
    """

    num_TP_in_pred = 0
    num_TP_in_true = 0
    num_instances_pred = 0
    num_instances_true = 0
    mask_class_label_pred = labels_pred == class_label
    mask_class_label_true = labels_true == class_label
    num_instances_pred = np.unique(instance_id_pred[labels_pred==class_label]).shape[0]
    num_instances_true = np.unique(instance_id_true[labels_true==class_label]).shape[0]
    intersection_sum = 0
    union_sum = 0

    if instance_id_pred.shape[0] >0:
        instance_id_pred_max = instance_id_pred.max()
    else:
        instance_id_pred_max = 0
    if instance_id_true.shape[0] >0:
        instance_id_true_max = instance_id_true.max()
    else:
        instance_id_true_max = 0

    intersection_sum = np.sum(np.logical_and(mask_class_label_pred, mask_class_label_true))
    union_sum = np.sum(np.logical_or(mask_class_label_pred, mask_class_label_true))
    num_TP_single_target = 0
    # calculate num_TP_in_pred (for precision)
    for i in np.arange(instance_id_pred_max + 1):
        indx_pred = np.logical_and(instance_id_pred == i, mask_class_label_pred)
        for j in np.arange(instance_id_true_max + 1):
            indx_true = np.logical_and(instance_id_true == j, mask_class_label_true)
            intersection = np.sum(np.logical_and(indx_pred, indx_true))
            union = np.sum(np.logical_or(indx_pred, indx_true))
            if intersection/union >= 0.5:
                num_TP_in_pred += 1
                break

    # calculate num_TP_in_true (for recall)
    for i in np.arange(instance_id_true_max + 1):
        indx_true = np.logical_and(instance_id_true == i, mask_class_label_true)
        for j in np.arange(instance_id_pred_max + 1):
            indx_pred = np.logical_and(instance_id_pred == j, mask_class_label_pred)
            intersection = np.sum(np.logical_and(indx_pred, indx_true))
            union = np.sum(np.logical_or(indx_pred, indx_true))
            num_true = np.sum(indx_true)
            num_pred = np.sum(indx_pred)

            if intersection/union >= 0.5:
                num_TP_in_true += 1
                if np.sum(indx_true) == 1:
                    num_TP_single_target +=1
                break

                

    return num_TP_in_pred, num_TP_in_true, num_instances_pred, num_instances_true, intersection_sum, union_sum, num_TP_single_target
    
def cal_precision_recall_all(labels_true, labels_pred, instance_id_true, instance_id_pred, frame_id, num_class = 4, targets_for_debug=None):
    """ 
    Parameters
    -------------
    labels_true: 1-d array
        The true labels of the targets of all the frames
    labels_pared: 1-d array
        The predicted labels of the targets of all the frames 
    instance_id_true:
        The instance ID of the targets of all frames from SSD bounding box
    instance_id_pred:
        The instance ID of the post-clustering output
    frame_id: 1-d array
        The frame_id of each target. The size of frame_id should be equal to labels_true and labels_pred 

    Returns
    -------------
    precision: 1-d array 
        the precision of each class
    recall: 1-d array
        the recall of each class

    """

    num_TP_in_pred = np.zeros(num_class)
    num_TP_in_true = np.zeros(num_class)
    intersection_sum = np.zeros(num_class)
    union_sum = np.zeros(num_class)
    num_instance_pred = np.zeros(num_class)
    num_instance_true = np.zeros(num_class)
    precision = np.zeros(num_class)
    recall = np.zeros(num_class)
    num_TP_single_target_total = 0
    pbar = tqdm.tqdm(total = frame_id.max()+1, desc='calculate precision and recall')
    for i in np.arange(0, frame_id.max()+1):
        pbar.update()
        # if i<100:
        #     continue
        labels_true_single = labels_true[frame_id==i]
        labels_pred_single = labels_pred[frame_id==i]
        instance_id_pred_single = instance_id_pred[frame_id==i]
        instance_id_true_single = instance_id_true[frame_id==i]
        for j in np.arange(1, num_class):
            num_TP_in_pred_single, num_TP_in_true_single, num_instance_pred_single, num_instance_true_single, intersection_single, union_single, num_TP_single_target = cal_precision_recall_single(labels_true_single, 
                                                                                                            labels_pred_single, 
                                                                                                            instance_id_true_single, 
                                                                                                            instance_id_pred_single, 
                                                                                                            class_label=j)
                                                                                                            # targets_for_debug=targets_for_debug[frame_id==i,:])
            # print("num_TP_in_pred_single",num_TP_in_pred_single)
            # print("num_TP_in_true_single", num_TP_in_true_single)

            num_TP_in_pred[j] += num_TP_in_pred_single
            num_TP_in_true[j] += num_TP_in_true_single
            num_instance_pred[j] += num_instance_pred_single
            num_instance_true[j] += num_instance_true_single
            intersection_sum[j] += intersection_single
            union_sum[j] += union_single
            num_TP_single_target_total += num_TP_single_target
    precision = num_TP_in_pred / num_instance_pred
    recall    = num_TP_in_true / num_instance_true

    return precision, recall, intersection_sum/union_sum, num_TP_single_target_total

eps_xy_list = [None, 0.5, 2 , 4]
eps_v_list = [None, 2, 1.6, 1]
min_targets_list = [None, 1, 2, 3]
def post_clustering(targets_xyv, labels_pred, frame_id, DBSCAN_eps = 1.1, DBSCAN_min_samples=1, algorithm=1, target_scores = None, filter_objects = False):
    """  
    Parameters:
    ------------------
    targets_xyv: 2-d array
        The x, y coordinates and velocity of targets
    labels_pred: 1-d array
        The predicted labels of the targets of all the frames 
    frame_id: 1-d array 
        The frame_id of each target. The size of frame_id should be equal to labels_pred and targets_rav
    """
    color_LUT = np.array(['c','g','r','b'])
    min_scores_list = []
    post_clst_id = -1 * np.ones(targets_xyv.shape[0])
    pbar = tqdm.tqdm(total = frame_id.max()+1, desc='post_clustering')
    t = 0
    debug_mode = True
    filter_objects = True
    for i in np.arange(0, frame_id.max() + 1):
        pbar.update()
        if debug_mode and i < 3803:
            continue
        targets_xyv_single = targets_xyv[frame_id==i, :]
        labels_pred_single = labels_pred[frame_id==i]
        targets_xyv_ped = targets_xyv_single[labels_pred_single==1, :]
        targets_xyv_biker = targets_xyv_single[labels_pred_single==2, :]
        targets_xyv_car = targets_xyv_single[labels_pred_single==3, :]
        targets_scores_single = target_scores[frame_id == i, :] / np.reshape(np.linalg.norm(target_scores[frame_id == i, :] , ord=2, axis=1), (-1, 1))
        # first time
        if targets_xyv_ped.shape[0] > 0:
            post_clst_id_ped = cylinder_cluster(targets_xyv_ped[:, :3], eps_xy = eps_xy_list[1], eps_v = eps_v_list[1], min_targets=min_targets_list[1])
            max_clst_id_ped = post_clst_id_ped.max()
        else:
            post_clst_id_ped = np.array([])
            max_clst_id_ped = -1
        if targets_xyv_biker.shape[0] > 1:
            post_clst_id_biker = cylinder_cluster(targets_xyv_biker[:, :3], eps_xy = eps_xy_list[2], eps_v = eps_v_list[2], min_targets=min_targets_list[2])
                
            max_clst_id_biker = max_clst_id_ped + 1 + post_clst_id_biker.max()
        else:
            post_clst_id_biker = -1 * np.ones([targets_xyv_biker.shape[0]])
            max_clst_id_biker = max_clst_id_ped 
        if targets_xyv_car.shape[0] > 2:
            post_clst_id_car = cylinder_cluster(targets_xyv_car[:, :3], eps_xy = eps_xy_list[3], eps_v = eps_v_list[3], min_targets=min_targets_list[3])
            
        else:
            post_clst_id_car = -1 * np.ones([targets_xyv_car.shape[0]])

        post_clst_id_biker[post_clst_id_biker>-1] = post_clst_id_biker[post_clst_id_biker>-1] + max_clst_id_ped+1
        post_clst_id_car[post_clst_id_car>-1] = post_clst_id_car[post_clst_id_car>-1] + max_clst_id_biker + 1

        post_clst_id_single = -1 * np.ones(np.sum(frame_id == i))
        post_clst_id_single[labels_pred_single==1] = post_clst_id_ped 
        post_clst_id_single[labels_pred_single==2] = post_clst_id_biker
        post_clst_id_single[labels_pred_single==3] = post_clst_id_car
        labels_pred_single[post_clst_id_single==-1] = 0
        if debug_mode:
            plt.figure(figsize = (16, 16))
            plt.title("Frame:{}".format(i))
            ax1 = plt.subplot(221)
            ax1.set_title("cluster before refinement")
            sc1 = ax1.scatter(targets_xyv_single[:, 1], targets_xyv_single[:, 0], c = post_clst_id_single, s = 10*post_clst_id_single+7)
            plt.colorbar(sc1, ax=ax1)
            ax1.set_xlim([-25, 25])
            ax1.set_ylim([0, 40])
            ax2 = plt.subplot(222)
            ax2.set_title("labels before refinement")
            ax2.scatter(targets_xyv_single[:, 1], targets_xyv_single[:, 0], c = color_LUT[labels_pred_single])
            ax2.set_xlim([-25, 25])
            ax2.set_ylim([0, 40])

        space_threshold = 1
        speed_threshold = [0, 3, 2, 1.2]
        score_threshold = 0.6

        if post_clst_id_single.shape[0] > 0 and filter_objects:
            min_dist_mat = 10 * np.ones([int(post_clst_id_single.max() + 1), int(post_clst_id_single.max() + 1)])
            min_v_diff_mat = 10 * np.ones([int(post_clst_id_single.max() + 1), int(post_clst_id_single.max() + 1)])
            object_label_list = 5 * np.ones(int(post_clst_id_single.max() + 1))
            for k in np.arange(int(post_clst_id_single.max() + 1)):
                for l in np.arange(int(post_clst_id_single.max() + 1)):
                    if k!=l and np.sum(post_clst_id_single == k) == 0 or np.sum(post_clst_id_single == l) == 0:
                        continue
                    object_label_list[k] = labels_pred_single[post_clst_id_single==k][0]
                    min_dist_pair = distance.cdist(targets_xyv_single[post_clst_id_single == k, :2], targets_xyv_single[post_clst_id_single == l, :2]).min()
                    min_dist_mat[k, l] = min_dist_pair 
                    min_v_diff_pair = distance.cdist(np.reshape(targets_xyv_single[post_clst_id_single == k, 2], (-1, 1)), np.reshape(targets_xyv_single[post_clst_id_single == l, 2], (-1, 1))).min()
                    min_v_diff_mat[k, l] = min_v_diff_pair
                    min_score_diff_pair = distance.cdist(targets_scores_single[post_clst_id_single == k, :], targets_scores_single[post_clst_id_single == l, :]).min()
                    if min_dist_pair < space_threshold:
                        label1 = labels_pred_single[post_clst_id_single == k][0] 
                        label2 = labels_pred_single[post_clst_id_single == l][0]
                        if (label1 == 2 and label2 == 3) or (label1 == 3 and label2 == 2):
                            min_scores_list.append(min_score_diff_pair)
                            # print(min_scores_list)
                            if min_v_diff_pair < speed_threshold[3]:
                                num_targets1 = np.sum(post_clst_id_single == k)
                                num_targets2 = np.sum(post_clst_id_single == l)
                                if num_targets1 > num_targets2:
                                    label_refine = label1
                                else:
                                    label_refine = label2 
                                if label_refine == 3 and min_score_diff_pair < score_threshold:
                                    post_clst_id_single[post_clst_id_single == k] = l
                                    labels_pred_single[post_clst_id_single == k] = label_refine
                                    labels_pred_single[post_clst_id_single == l] = label_refine
                        elif (label1 == 1 and label2 == 3) or (label1 == 3 and label2 == 1):
                            if min_v_diff_pair < speed_threshold[3]:
                                num_targets1 = np.sum(post_clst_id_single == k)
                                num_targets2 = np.sum(post_clst_id_single == l)
                                if num_targets1 > num_targets2:
                                    label_refine = label1
                                else:
                                    label_refine = label2 
                                if label_refine == 3  and min_score_diff_pair < score_threshold:
                                    post_clst_id_single[post_clst_id_single == k] = l
                                    labels_pred_single[post_clst_id_single == k] = label_refine
                                    labels_pred_single[post_clst_id_single == l] = label_refine
                        elif (label1 == 1 and label2 == 2) or (label1 == 2 and label2 == 1):
                            if min_v_diff_pair < speed_threshold[2]:
                                num_targets1 = np.sum(post_clst_id_single == k)
                                num_targets2 = np.sum(post_clst_id_single == l)
                                if num_targets1 > num_targets2:
                                    label_refine = label1
                                else:
                                    label_refine = label2 
                                if label_refine == 2 and min_score_diff_pair < score_threshold:
                                    post_clst_id_single[post_clst_id_single == k] = l
                                    labels_pred_single[post_clst_id_single == k] = label_refine
                                    labels_pred_single[post_clst_id_single == l] = label_refine
            num_obj_around = np.sum(np.logical_and(min_dist_mat < space_threshold, min_v_diff_mat < speed_threshold[3]), axis = 1)
            id_list_of_cars_surrounded_by_a_lot_of_bikers = np.nonzero(np.logical_and(num_obj_around>2, object_label_list == 3))
            for id_of_cars_surrounded_by_a_lot_of_bikers in id_list_of_cars_surrounded_by_a_lot_of_bikers:
                labels_pred_single[post_clst_id_single == id_of_cars_surrounded_by_a_lot_of_bikers] = 2

        # second time                     
        targets_xyv_ped = targets_xyv_single[labels_pred_single==1, :]
        targets_xyv_biker = targets_xyv_single[labels_pred_single==2, :]
        targets_xyv_car = targets_xyv_single[labels_pred_single==3, :]
        if targets_xyv_ped.shape[0] > 0:
            post_clst_id_ped = cylinder_cluster(targets_xyv_ped[:, :3], eps_xy = eps_xy_list[1], eps_v = eps_v_list[1], min_targets=min_targets_list[1])
            max_clst_id_ped = post_clst_id_ped.max()
        else:
            post_clst_id_ped = -1 * np.ones([targets_xyv_ped.shape[0]])
            max_clst_id_ped = -1
        if targets_xyv_biker.shape[0] > 1:
            post_clst_id_biker = cylinder_cluster(targets_xyv_biker[:, :3], eps_xy = eps_xy_list[2], eps_v = eps_v_list[2], min_targets=min_targets_list[2] )
            max_clst_id_biker = max_clst_id_ped + 1 + post_clst_id_biker.max()
        else:
            post_clst_id_biker = -1 * np.ones([targets_xyv_biker.shape[0]])
            max_clst_id_biker = max_clst_id_ped 
        if targets_xyv_car.shape[0] > 2:
            post_clst_id_car = cylinder_cluster(targets_xyv_car[:, :3], eps_xy = eps_xy_list[3], eps_v = eps_v_list[3], min_targets=min_targets_list[3])
            
        else:
            post_clst_id_car = -1 * np.ones([targets_xyv_car.shape[0]])

        post_clst_id_biker[post_clst_id_biker>-1] = post_clst_id_biker[post_clst_id_biker>-1] + max_clst_id_ped+1
        post_clst_id_car[post_clst_id_car>-1] = post_clst_id_car[post_clst_id_car>-1] + max_clst_id_biker + 1

        post_clst_id_single = -1 * np.ones(np.sum(frame_id == i))
        post_clst_id_single[labels_pred_single==1] = post_clst_id_ped 
        post_clst_id_single[labels_pred_single==2] = post_clst_id_biker
        post_clst_id_single[labels_pred_single==3] = post_clst_id_car
        labels_pred_single[post_clst_id_single==-1] = 0

        post_clst_id[frame_id==i] = post_clst_id_single
        labels_pred[frame_id==i] = labels_pred_single

    min_scores_list = np.array(min_scores_list)
    return post_clst_id, labels_pred 
def cal_f1(precision, recall):

    return 2*precision*recall/(precision+recall)

if __name__ == "__main__":
    """ Test the post_clustering->precision & recall pipeline """
    BASE_DIR = '/content/radar_cnn'
    result_DIR = osp.join(BASE_DIR, 'results', 'RTCtrain_info')
    config_list = os.listdir(result_DIR)
    config_list.sort()
    RTCnet_result_info_path = osp.join(result_DIR, config_list[-1])
    speed_threshold = 0.3
    speed_threshold_to_change_label = 0
    show_RTC_result = True
    with open(RTCnet_result_info_path, 'r') as f:
        RTCnet_result_info = json.load(f)


    if show_RTC_result:
        result_path = RTCnet_result_info["result_folder"]

        data_path = RTCnet_result_info["data_path"]
       
        labels_pred = np.load(osp.join(result_path, "pred_labels_test.npy"))
        labels_true = np.load(osp.join(result_path, "true_labels_test.npy"))
        target_scores = np.load(osp.join(result_path, "final_score.npy"))
        frame_id = np.load(osp.join(data_path, "test", "frame_id.npy"))
        instance_id_true = np.load(osp.join(data_path, "instance_id_test.npy"))
        targets_rav = np.load(osp.join(data_path, "test", "features.npy"))[:, :3]


        targets_xyv = np.zeros(targets_rav.shape)
        targets_xyv[:,0] = targets_rav[:,0]*np.cos(targets_rav[:,1])
        targets_xyv[:,1] = targets_rav[:,0]*np.sin(targets_rav[:,1])
        targets_xyv[:,2] = targets_rav[:,2]
        targets_v = np.abs(targets_rav[:, 2])
        labels_pred[targets_v < speed_threshold_to_change_label] = 0
        labels_pred = labels_pred[targets_v > speed_threshold]
        labels_true = labels_true[targets_v > speed_threshold]
        target_scores = target_scores[targets_v > speed_threshold]
        frame_id = frame_id[targets_v > speed_threshold]
        targets_rav = targets_rav[targets_v>speed_threshold, :]
        targets_xyv = targets_xyv[targets_v > speed_threshold, :]
        
        instance_id_true = instance_id_true[targets_v > speed_threshold]
        instance_id_pred, labels_pred = post_clustering(targets_xyv, labels_pred, frame_id, target_scores = target_scores)
        precision_RTC, recall_RTC, IoU_RTC, num_TP_single_target_total = cal_precision_recall_all(labels_true, labels_pred, instance_id_true, instance_id_pred, frame_id, targets_for_debug=targets_xyv)
 
        print("precision_RTC", precision_RTC, "recall_RTC", recall_RTC)
        np.savetxt(osp.join(result_path, "precision_RTC.csv"), precision_RTC, delimiter=",")
        np.savetxt(osp.join(result_path, "recall_RTC.csv"), recall_RTC, delimiter=",")
        np.savetxt(osp.join(result_path, "F1score.csv"), cal_f1(precision_RTC, recall_RTC), delimiter=",")
        np.savetxt(osp.join(result_path, "IoU_RTC.csv"), IoU_RTC, delimiter=",")
        np.save(osp.join(result_path, "pred_labels_refine.npy"), labels_pred)
        print("number of TP object with 1 target:", num_TP_single_target_total)
        print("f1 RTC: ", cal_f1(precision_RTC, recall_RTC))
        print("IoU RTC:", IoU_RTC)


post_clustering: 100%|██████████| 200/200.0 [00:00<00:00, 251005.63it/s]
calculate precision and recall:   0%|          | 0/200.0 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:153: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:154: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:156: RuntimeWarning: invalid value encountered in true_divide
calculate precision and recall: 100%|██████████| 200/200.0 [00:00<00:00, 3970.37it/s]

precision_RTC [nan  0.  0. nan] recall_RTC [nan  0.  0.  0.]
number of TP object with 1 target: 0
f1 RTC:  [nan nan nan nan]
IoU RTC: [       nan 0.10643185 0.16538343 0.        ]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:334: RuntimeWarning: invalid value encountered in true_divide
